In [1]:
# !tensorboard --logdir=runs

# Import module

In [2]:
from torch.utils.tensorboard import SummaryWriter

# Define writer

In [3]:
writer = SummaryWriter("runs/2MNIST")

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
import torch.nn.functional as F

In [5]:

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001


In [6]:

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# Create images

In [7]:

examples = iter(test_loader)
example_data, example_targets = examples.next()

# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(example_data[i][0], cmap='gray')
# plt.show()



# Add images to the tensorboard

In [8]:
img_grid = torchvision.utils.make_grid(example_data)
writer.add_image("mnist_images", img_grid)
# writer.close()

In [9]:

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out


In [10]:

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


# Add model graph to the tensorboard

In [11]:
writer.add_graph(model, example_data.reshape(-1, 28 * 28).to(device))
# writer.close()

# Modify thain loop to check loss in tensorboard

In [12]:

# Train the model
n_total_steps = len(train_loader)
running_loss = 0.0
running_correct_pred = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # need to tensorboard
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        running_correct_pred += (predicted == labels).sum().item()
        
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            writer.add_scalar("training loss", running_loss / 100, epoch * n_total_steps + i)
            writer.add_scalar("accuracy", running_correct_pred / 100, epoch * n_total_steps + i)
            running_loss = 0.0
            running_correct_pred = 0

Epoch [1/2], Step [100/600], Loss: 0.3057
Epoch [1/2], Step [200/600], Loss: 0.3061
Epoch [1/2], Step [300/600], Loss: 0.1433
Epoch [1/2], Step [400/600], Loss: 0.1880
Epoch [1/2], Step [500/600], Loss: 0.2172
Epoch [1/2], Step [600/600], Loss: 0.1609
Epoch [2/2], Step [100/600], Loss: 0.1194
Epoch [2/2], Step [200/600], Loss: 0.0981
Epoch [2/2], Step [300/600], Loss: 0.1362
Epoch [2/2], Step [400/600], Loss: 0.0604
Epoch [2/2], Step [500/600], Loss: 0.0860
Epoch [2/2], Step [600/600], Loss: 0.0783


# Evatuation with tensorboard

In [14]:
labels = []
preds = []


# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels1 in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels1 = labels1.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels1.size(0)
        n_correct += (predicted == labels1).sum().item()
        
        class_preds = [F.softmax(output, dim=0) for output in outputs]
        
        preds.append(class_preds)
        labels.append(predicted)
    
    # Reshaping 
    preds = torch.cat([torch.stack(batch) for batch in preds])
    labels = torch.cat(labels)



    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')
    
    classes = range(10)
    for i in classes:
        labels_i = labels == i
        preds_i = preds[:, i]
        writer.add_pr_curve(str(i), labels_i, preds_i, global_step=0)
    writer.close()


Accuracy of the network on the 10000 test images: 97.08 %
